In [23]:
import pandas as pd
import numpy as np

In [24]:
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, recall_score

I have modeified the target values if "Financial Distress" if it is greater than -0.50 the company should be considered as healthy (0). Otherwise, it would be regarded as financially distressed (1). 

In [25]:
data=pd.read_csv("Financial Distress.csv")

In [26]:
data.head()

,Company,Time,Financial Distress,x1,x2,x3,x4,x5,x6,x7,...,x74,x75,x76,x77,x78,x79,x80,x81,x82,x83
0,1,1,0.010636,1.2810,0.022934,0.87454,1.21640,0.060940,0.188270,0.52510,...,85.437,27.07,26.102,16.000,16.0,0.2,22,0.060390,30,49
1,1,2,-0.455970,1.2700,0.006454,0.82067,1.00490,-0.014080,0.181040,0.62288,...,107.090,31.31,30.194,17.000,16.0,0.4,22,0.010636,31,50
2,1,3,-0.325390,1.0529,-0.059379,0.92242,0.72926,0.020476,0.044865,0.43292,...,120.870,36.07,35.273,17.000,15.0,-0.2,22,-0.455970,32,51
3,1,4,-0.566570,1.1131,-0.015229,0.85888,0.80974,0.076037,0.091033,0.67546,...,54.806,39.80,38.377,17.167,16.0,5.6,22,-0.325390,33,52
4,2,1,1.357300,1.0623,0.107020,0.81460,0.83593,0.199960,0.047800,0.74200,...,85.437,27.07,26.102,16.000,16.0,0.2,29,1.251000,7,27


In [27]:
data.columns

Index(['Company', 'Time', 'Financial Distress', 'x1', 'x2', 'x3', 'x4', 'x5',
       'x6', 'x7', 'x8', 'x9', 'x10', 'x11', 'x12', 'x13', 'x14', 'x15', 'x16',
       'x17', 'x18', 'x19', 'x20', 'x21', 'x22', 'x23', 'x24', 'x25', 'x26',
       'x27', 'x28', 'x29', 'x30', 'x31', 'x32', 'x33', 'x34', 'x35', 'x36',
       'x37', 'x38', 'x39', 'x40', 'x41', 'x42', 'x43', 'x44', 'x45', 'x46',
       'x47', 'x48', 'x49', 'x50', 'x51', 'x52', 'x53', 'x54', 'x55', 'x56',
       'x57', 'x58', 'x59', 'x60', 'x61', 'x62', 'x63', 'x64', 'x65', 'x66',
       'x67', 'x68', 'x69', 'x70', 'x71', 'x72', 'x73', 'x74', 'x75', 'x76',
       'x77', 'x78', 'x79', 'x80', 'x81', 'x82', 'x83'],
      dtype='object')

In [28]:
data['Financial Distress'].value_counts()

0.00000    12
1.82880     2
1.34620     2
1.22410     2
1.80660     2
           ..
0.30504     1
2.74930     1
0.91136     1
0.46971     1
0.47159     1
Name: Financial Distress, Length: 3610, dtype: int64

In [22]:
X, y = data.loc[:,data.columns!='Financial Distress'], data.loc[:,'Financial Distress'].values
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=123,stratify=data['Financial Distress'])

ValueError: The least populated class in y has only 1 member, which is too few. The minimum number of groups for any class cannot be less than 2.

In [8]:
# We have used stratified above to split the data distribution in equal manner
print(pd.value_counts(y_train)/y_train.size * 100)
print(pd.value_counts(y_test)/y_test.size * 100)

0    96.303502
1     3.696498
dtype: float64
0    96.279492
1     3.720508
dtype: float64


Accuracy is not the best metric to use when evaluating imbalanced datasets as it can be misleading. Metrics that can provide better insight include:

    Confusion Matrix: a talbe showing correct predictions and types of incorrect predictions.
    Precision: the number of true positives divided by all positive predictions. Precision is also called Positive Predictive Value. It is a measure of a classifier's exactness. Low precision indicates a high number of false positives.
    Recall: the number of true positives divided by the number of positive values in the test data. Recall is also called Sensitivity or the True Positive Rate. It is a measure of a classifier's completeness. Low recall indicates a high number of false negatives.
    F1: Score: the weighted average of precision and recall.

Since our main objective with the dataset is to prioritize accuraltely classifying financial instability cases the recall score can be considered our main metric to use for evaluating outcomes.

## Trying different algorithm

In [9]:
from sklearn.ensemble import RandomForestClassifier
# train model
rfc = RandomForestClassifier(n_estimators=10).fit(X_train, y_train)

# predict on test set
rfc_pred = rfc.predict(X_test)

accuracy_score(y_test, rfc_pred)

0.9618874773139746

In [10]:
# f1 score
f1_score(y_test, rfc_pred)


0.16

In [11]:
# confusion matrix
pd.DataFrame(confusion_matrix(y_test, rfc_pred))

,0,1
0,1056,5
1,37,4


In [12]:
# recall score
recall_score(y_test, rfc_pred)

0.0975609756097561

Oversampling Minority Class

Oversampling can be defined as adding more copies of the minority class. Oversampling can be a good choice when you don't have a ton of data to work with. A con to consider when undersampling is that it can cause overfitting and poor generalization to your test set.

We will use the resampling module from Scikit-Learn to randomly replicate samples from the minority class.
Important Note

Always split into test and train sets BEFORE trying any resampling techniques! Oversampling before splitting the data can allow the exact same observations to be present in both the test and train sets! This can allow our model to simply memorize specific data points and cause overfitting.

In [13]:
from sklearn.utils import resample


In [14]:
X['class']=y

In [15]:
X.columns

Index(['Company', 'Time', 'x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9',
       'x10', 'x11', 'x12', 'x13', 'x14', 'x15', 'x16', 'x17', 'x18', 'x19',
       'x20', 'x21', 'x22', 'x23', 'x24', 'x25', 'x26', 'x27', 'x28', 'x29',
       'x30', 'x31', 'x32', 'x33', 'x34', 'x35', 'x36', 'x37', 'x38', 'x39',
       'x40', 'x41', 'x42', 'x43', 'x44', 'x45', 'x46', 'x47', 'x48', 'x49',
       'x50', 'x51', 'x52', 'x53', 'x54', 'x55', 'x56', 'x57', 'x58', 'x59',
       'x60', 'x61', 'x62', 'x63', 'x64', 'x65', 'x66', 'x67', 'x68', 'x69',
       'x70', 'x71', 'x72', 'x73', 'x74', 'x75', 'x76', 'x77', 'x78', 'x79',
       'x80', 'x81', 'x82', 'x83', 'class'],
      dtype='object')

In [16]:
# separate minority and majority classes
not_distress = X[X['class']==0]
distress = X[X['class']==1]

# upsample minority
fraud_upsampled = resample(distress,
                          replace=True, # sample with replacement
                          n_samples=len(not_distress), # match number in majority class
                          random_state=27) # reproducible results

# combine majority and upsampled minority
upsampled = pd.concat([not_distress, fraud_upsampled])

# check new class counts
upsampled['class'].value_counts()

1    3536
0    3536
Name: class, dtype: int64

In [17]:
# trying logistic regression again with the balanced dataset
y_train = upsampled['class']
X_train = upsampled.drop('class', axis=1)

upsampled = LogisticRegression(solver='liblinear').fit(X_train, y_train)

upsampled_pred = upsampled.predict(X_test)


/opt/conda/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [18]:
# Checking accuracy
accuracy_score(y_test, upsampled_pred)

0.8112522686025408

In [19]:
# f1 score
f1_score(y_test, upsampled_pred)

0.22388059701492535

In [20]:
# confusion matrix
pd.DataFrame(confusion_matrix(y_test, upsampled_pred))

,0,1
0,864,197
1,11,30


In [21]:
recall_score(y_test, upsampled_pred)

0.7317073170731707

Undersampling Majority Class

Undersampling can be defined as removing some observations of the majority class. Undersampling can be a good choice when you have a ton of data -think millions of rows. But a drawback to undersampling is that we are removing information that may be valuable.

We will again use the resampling module from Scikit-Learn to randomly remove samples from the majority class.

In [22]:
# downsample majority
not_fraud_downsampled = resample(not_distress,
                                replace = False, # sample without replacement
                                n_samples = len(distress), # match minority n
                                random_state = 27) # reproducible results

# combine minority and downsampled majority
downsampled = pd.concat([not_fraud_downsampled, distress])

# checking counts
downsampled['class'].value_counts()

1    136
0    136
Name: class, dtype: int64

In [23]:
y_train = downsampled['class']
X_train = downsampled.drop('class', axis=1)

undersampled = LogisticRegression(solver='liblinear').fit(X_train, y_train)

undersampled_pred = undersampled.predict(X_test)

In [24]:
# Checking accuracy
accuracy_score(y_test, undersampled_pred)

0.7468239564428312

In [25]:
# f1 score
f1_score(y_test, undersampled_pred)

0.2140845070422535

In [26]:
# confusion matrix
pd.DataFrame(confusion_matrix(y_test, undersampled_pred))

,0,1
0,785,276
1,3,38


In [27]:
recall_score(y_test, undersampled_pred)

0.926829268292683

Generate Synthetic Samples

SMOTE or Synthetic Minority Oversampling Technique is a popular algorithm to creates sythetic observations of the minority class.

In [28]:
from imblearn.over_sampling import SMOTE

# Separate input features and target
y = data['Financial Distress']
X = data.drop('Financial Distress', axis=1)

# setting up testing and training sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=27)

sm = SMOTE(random_state=27, ratio=1.0)
X_train, y_train = sm.fit_sample(X_train, y_train)

Using TensorFlow backend.


In [29]:
smote = LogisticRegression(solver='liblinear').fit(X_train, y_train)

smote_pred = smote.predict(X_test)

# Checking accuracy
accuracy_score(y_test, smote_pred)


/opt/conda/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.7690631808278867

In [30]:
# f1 score
f1_score(y_test, smote_pred, average='weighted')


0.8439387964406211

In [31]:
# confustion matrix
pd.DataFrame(confusion_matrix(y_test, smote_pred))


,0,1
0,683,206
1,6,23


In [32]:
recall_score(y_test, smote_pred)


0.7931034482758621

Over here undersampling worked the best but many other algorithms like LightGBM, XGBoost will work better which I will try to cover later in this notebook.